In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import codecs
import gc
import json
import time
import sklearn
from sklearn import *
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.base import TransformerMixin

DATA_DIR = 'washed'
FILE_NAME = 'train_wfd.csv'
INPUT_FILE_PATH = os.path.join(DATA_DIR, FILE_NAME)

COLUMN_NAMES = ['distance', 'pickup_longitude', 
                'dropoff_longitude', 'pickup_hour', 'pickup_year']

CHUNKSIZE = 500000

JFK_RANGE = (-73.8250, -73.7746, 40.6397, 40.7121)

def get_jfk_mask(d):
    return (d.pickup_longitude >= JFK_RANGE[0]) & (d.pickup_longitude <= JFK_RANGE[1]) & \
           (d.pickup_latitude >= JFK_RANGE[2]) & (d.pickup_latitude <= JFK_RANGE[3]) | \
           (d.dropoff_longitude >= JFK_RANGE[0]) & (d.dropoff_longitude <= JFK_RANGE[1]) & \
           (d.dropoff_latitude >= JFK_RANGE[2]) & (d.dropoff_latitude <= JFK_RANGE[3])

df = pd.read_csv(INPUT_FILE_PATH, nrows=CHUNKSIZE)
df = df.assign(distance=((df.pickup_longitude-df.dropoff_longitude).pow(2) + (df.pickup_latitude-df.dropoff_latitude).pow(2)).pow(0.5))
df = df.assign(pickup_hour=(df.pickup_daytime/3600).round())

testdf = pd.read_csv('washed\\test_fd.csv')
testdf = testdf.assign(distance=((testdf.pickup_longitude-testdf.dropoff_longitude).pow(2) + (testdf.pickup_latitude-testdf.dropoff_latitude).pow(2)).pow(0.5))
testdf = testdf.assign(pickup_hour=(testdf.pickup_daytime/3600).round())

# jfkmask = get_jfk_mask(testdf)

# print (np.sum(jfkmask))

# exit(1)

X = df[COLUMN_NAMES]
y = df.fare_amount

train_in, test_in, train_f, test_f = sklearn.model_selection.train_test_split(
    X, y, train_size=0.8, random_state=1)

F:\DS_WORKSPACE\NYC_TEXI\venv\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [2]:
def write_predicted(d, modelname, datasize):
    tstring = str(time.time())
    d.to_csv('predicted\\{}_predicted.csv'.format(tstring), index=False)
    with open('predicted\\{}_feature.txt'.format(tstring), 'w') as f:
        f.write(', '.join(COLUMN_NAMES)+
                '\n{} data points used.'.format(datasize)+
                '\n{} model'.format(modelname))
        f.close()

In [3]:
lr = sklearn.linear_model.LinearRegression()
lr.fit(train_in, train_f)
# pY = lr.predict(test_in)
lr_mse = mean_squared_error(test_in, test_f)
print ("MSE: {}".format(lr_mse))
pY = lr.predict(testdf[COLUMN_NAMES])

predicted = testdf[['key']].copy()
predicted = predicted.assign(fare_amount=pd.Series(pY))

write_predicted(predicted, 'simplest linear regression', CHUNKSIZE)

0.7687661903757836


In [4]:
from sklearn.preprocessing import PolynomialFeatures
polyr = sklearn.linear_model.LinearRegression()

poly_featurizer = PolynomialFeatures(degree=2)

train_poly = poly_featurizer.fit_transform(train_in)
test_poly = poly_featurizer.transform(test_in)
polyr.fit(train_poly, train_f)
poly_mse = mean_squared_error(test_poly, test_f)
print ("MSE: {}".format(poly_mse))

pred_poly = poly_featurizer.fit_transform(testdf[COLUMN_NAMES])
poly_pY = polyr.predict(pred_poly)

poly_predicted = testdf[['key']].copy()
poly_predicted = poly_predicted.assign(fare_amount=pd.Series(poly_pY))
write_predicted(poly_predicted, 'polynomial regression', CHUNKSIZE)


0.7761509513971669


In [12]:
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
gboost_reg = ensemble.GradientBoostingRegressor(**params)

gboost_reg.fit(train_in, train_f)
g_mse = mean_squared_error(test_f, gboost_reg.predict(test_in))
print("MSE: {}".format(g_mse))

g_pY = gboost_reg.predict(testdf[COLUMN_NAMES])
g_predicted = testdf[['key']].copy()
g_predicted = g_predicted.assign(fare_amount=pd.Series(g_pY))
write_predicted(g_predicted, 'gboost regression', CHUNKSIZE)

MSE: 18.4659502083883


In [14]:
xg_r = xgb.XGBRegressor(max_depth=3, n_estimators=500, learning_rate=0.01)\
    .fit(train_in, train_f)
xg_mse = mean_squared_error(test_f, xg_r.predict(test_in))
print("MSE: {}".format(xg_mse))
xg_pY = xg_r.predict(testdf[COLUMN_NAMES])
xg_predicted = testdf[['key']].copy()
xg_predicted = xg_predicted.assign(fare_amount=pd.Series(xg_pY))
write_predicted(xg_predicted, 'xgboost regression', CHUNKSIZE)